In [2]:
from pandas import read_csv
from aws_s3 import AwsS3
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

class Crawler():
    
    def get_matches_report_detail() -> str:

        options = webdriver.ChromeOptions()
        options.add_experimental_option('excludeSwitches', ['enable-logging'])
        driver = webdriver.Remote('http://127.0.0.1:4444/wd/hub', options = options)

        path_write = 'raw/trackergg/matches_report_details/'

        matches = read_csv("matches.csv")
        matches = matches['match_id'].to_list()

        for matche in matches:
            
            driver.get('https://api.tracker.gg/api/v2/valorant/standard/matches/{}'.format(matche))
            
            data_pre = driver.find_element('xpath', '//pre').text

            time.sleep(5)

            file_format = '.txt'
            
            AwsS3.upload_file(data_pre, path_write, file_format)

        driver.quit()

        # return data_pre

In [ ]:
Crawler.get_matches_report_detail()

In [3]:
import boto3
import os
from dotenv import load_dotenv
from datetime import datetime
from botocore.exceptions import ClientError
import logging

load_dotenv()

AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_S3_BUCKET = os.getenv("AWS_S3_BUCKET")


class AwsS3():

    
    def upload_file(data : object, path : str, file_format) -> bool:

        """
            Upload a file to an S3 bucket
            :param file_name: File to upload
            :param bucket: Bucket to upload to
            :param object_name: S3 object name. If not specified then file_name is used
            :return: True if file was uploaded, else False
        """

        date = datetime.now().strftime("_%Y%m%d_%H%M%S")
        file_name = 'valorant_reports{}{}'.format(date, file_format)
        input = path + file_name

        
        s3 = boto3.client("s3", aws_access_key_id = AWS_ACCESS_KEY_ID, aws_secret_access_key = AWS_SECRET_ACCESS_KEY)

        try:
            s3.put_object(Bucket = AWS_S3_BUCKET, Body = data, Key = input)

        except ClientError as e:
            logging.error(e)

            return False

        return True

    
    def get_file(path : str, file_name : str) -> str:

        """
            Get a file to an S3 bucket
            :param Path: Path to get
            :param bucket: Bucket to upload to
            :param object_name: S3 object name. If not specified then file_name is used
            :return: True if file was uploaded, else False
        """
        s3 = boto3.client('s3')
        
        try:

            response = s3.get_object(Bucket = AWS_S3_BUCKET, Key = file_name)
            data = response['Body'].read()
            data_str = data.decode('utf-8')

        except ClientError as e:
            logging.error(e)


        return data_str
        

    def get_files_list(path_read : str) -> list:

        s3 = boto3.resource('s3')
        bucket = s3.Bucket(AWS_S3_BUCKET)
        files_list = bucket.objects.filter(Prefix = path_read)
        files_list = list(files_list)
        
        if len(files_list) > 1: 
            del files_list[0]
        else:
            pass

        return files_list

In [4]:
#from aws_s3 import AwsS3
import pandas as pd
import json


class DataCleaner():
    
    def metadata_df():
        """Metadata da partida completa, dados diretos referentes a aquela partida, por exemplo, mapa, n de rounds, ranqueada ou nao e etc.
        """
        path_read = 'raw/trackergg/matches_report_details/'
        path_write = 'cleaned/trackergg/matches_report_details/'     

        files = AwsS3.get_files_list(path_read)

        data = []

        for file in files:
            data_json = json.loads(AwsS3.get_file(path_read, file.key))
            metadata : dict = data_json["data"]["metadata"]
            metadata['match_id'] = data_json['data']["attributes"]["id"]
            data.append(metadata)
        
            
        return data
           
    def player_round_df():
        """Stats basicas do jogador por round, kills, mortes, assistencias, etc.
        """

        path_read = 'raw/trackergg/matches_report_details/'
        path_write = 'cleaned/trackergg/matches_report_details/'
        
        files = AwsS3.get_files_list(path_read)

        data = []
        for file in files:
            data_json = json.loads(AwsS3.get_file(path_read, file.key))
            segments = data_json['data']['segments']

            for segment in segments:
                if segment['type'] == 'player-round':
                    segment_dict = {}
                    attributes = segment['attributes']
                    segment_dict.update(attributes)
                    metadata = segment['metadata']
                    segment_dict.update(metadata)
                    segment_stats = segment['stats']
                    segment_stats_dict = {}
                    for stat, stat_data in segment_stats.items():
                        stat_keys = stat_data.keys()
                        stat_columns = [f'{stat}_{col}' for col in stat_keys]
                        stat_values = stat_data.values()
                        _stat_dict = {k: v for k, v in zip(stat_columns, stat_values)}
                        segment_stats_dict.update(_stat_dict)
                    segment_dict.update(segment_stats_dict)
                    segment_dict['match_id'] = data_json['data']["attributes"]["id"]
                    data.append(segment_dict)
                    
        return data
    
    def player_round_damage_df():
        """Stat do dano causado por aquele jogador em certo jogador. Exemplo: Quantidade de dano que o jogador A causou no jogador B, uma relação de 1 pra 1 por linha.
        """

        path_read = 'raw/trackergg/matches_report_details/'
        path_write = 'cleaned/trackergg/matches_report_details/'
            
        df_aux = pd.DataFrame()

        files = AwsS3.get_files_list(path_read)

        data = []
        for file in files:
            data_json = json.loads(AwsS3.get_file(path_read, file.key))
            segments = data_json['data']['segments']

            for segment in segments:
                if segment['type'] == 'player-round-damage':
                    segment_dict = {}
                    attributes = segment['attributes']
                    segment_dict.update(attributes)
                    segment_stats = segment['stats']
                    segment_stats_dict = {}
                    for stat, stat_data in segment_stats.items():
                        stat_keys = stat_data.keys()
                        stat_columns = [f'{stat}_{col}' for col in stat_keys]
                        stat_values = stat_data.values()
                        _stat_dict = {k: v for k, v in zip(stat_columns, stat_values)}
                        segment_stats_dict.update(_stat_dict)
                    segment_dict.update(segment_stats_dict)
                    segment_dict['match_id'] = data_json['data']["attributes"]["id"]
                    data.append(segment_dict)

            
                
        return data


    def player_summmary_df():

        path_read = 'raw/trackergg/matches_report_details/'
        path_write = 'cleaned/trackergg/matches_report_details/'
            
        df_aux = pd.DataFrame()

        files = AwsS3.get_files_list(path_read)

        data = []
        for file in files:
            data_json = json.loads(AwsS3.get_file(path_read, file.key))
            segments = data_json['data']['segments']

            for segment in segments:
                if segment['type'] == 'player-summary':
                    segment_dict = {}
                    attributes = segment['attributes']
                    segment_dict.update(attributes)
                    metadata = segment['metadata']
                    segment_dict.update(metadata)
                    segment_stats = segment['stats']
                    segment_stats_dict = {}
                    for stat, stat_data in segment_stats.items():
                        try:
                            stat_keys = stat_data.keys()
                            stat_columns = [f'{stat}_{col}' for col in stat_keys]
                            stat_values = stat_data.values()
                            _stat_dict = {k: v for k, v in zip(stat_columns, stat_values)}
                            segment_stats_dict.update(_stat_dict)
                        except AttributeError:
                            """Tratativa de excecao para o caso em que nao tivermos alguma coluna dentro dos status"""
                            pass
                    segment_dict.update(segment_stats_dict)
                    segment_dict['match_id'] = data_json['data']["attributes"]["id"]
                    data.append(segment_dict)

                
        return data
    
    def player_round_kills_df():

        path_read = 'raw/trackergg/matches_report_details/'
        path_write = 'cleaned/trackergg/matches_report_details/'
                
        df_aux = pd.DataFrame()

        files = AwsS3.get_files_list(path_read)

        data = []
        for file in files:
            data_json = json.loads(AwsS3.get_file(path_read, file.key))
            segments = data_json['data']['segments']

            for segment in segments:
                if segment['type'] == 'player-round-kills':
                    segment_dict = {}
                    attributes = segment['attributes']
                    segment_dict.update(attributes)
                    metadata = segment['metadata']
                    segment_metadata_dict = {}
                    for metadata_iter, metadata_data in metadata.items():
                        try:
                            metadata_keys = metadata_data.keys()
                            metadata_columns = [f'{metadata_iter}_{col}' for col in metadata_keys]
                            metadata_values = metadata_data.values()
                            _metadata_dict = {k: v for k, v in zip(metadata_columns, metadata_values)}
                            segment_metadata_dict.update(_metadata_dict)
                        except AttributeError:
                            """Tratativa de excecao para o caso em que nao tivermos alguma coluna dentro dos status"""
                            metadata_weaponImageUrl = metadata['weaponImageUrl']
                            metadata_weaponName = metadata['weaponName']
                            metadata_weaponCategory = metadata['weaponCategory']
                            metadata_gameTime = metadata['gameTime']
                            metadata_roundTime = metadata['roundTime']
                            pass
                        segment_stats_damage = segment['stats']['damage']
                        segment_metadata_dict.update(segment_stats_damage)
                    segment_dict.update(segment_metadata_dict)
                    segment_dict['match_id'] = data_json['data']["attributes"]["id"]
                    data.append(segment_dict)
                        
        return data
       
    def all_dfs():
        path_read = 'raw/trackergg/matches_report_details/'
        path_write = 'cleaned/trackergg/matches_report_details/'     

        files = AwsS3.get_files_list(path_read)

        metadata_dict_list = []
        player_round_dict_list = []
        player_round_damage_dict_list = []
        player_summary_dict_list = []
        player_round_kills = []

        for file in files:
            data_json = json.loads(AwsS3.get_file(path_read, file.key))
            metadata : dict = data_json["data"]["metadata"]
            metadata['match_id'] = data_json['data']["attributes"]["id"]
            metadata_dict_list.append(metadata)

            segments = data_json['data']['segments']

            for segment in segments:
                if segment['type'] == 'player-round':
                    segment_dict = {}
                    attributes = segment['attributes']
                    segment_dict.update(attributes)
                    metadata = segment['metadata']
                    segment_dict.update(metadata)
                    segment_stats = segment['stats']
                    segment_stats_dict = {}
                    for stat, stat_data in segment_stats.items():
                        stat_keys = stat_data.keys()
                        stat_columns = [f'{stat}_{col}' for col in stat_keys]
                        stat_values = stat_data.values()
                        _stat_dict = {k: v for k, v in zip(stat_columns, stat_values)}
                        segment_stats_dict.update(_stat_dict)
                    segment_dict.update(segment_stats_dict)
                    segment_dict['match_id'] = data_json['data']["attributes"]["id"]
                    player_round_dict_list.append(segment_dict)

                elif segment['type'] == 'player-round-damage':
                    segment_dict = {}
                    attributes = segment['attributes']
                    segment_dict.update(attributes)
                    segment_stats = segment['stats']
                    segment_stats_dict = {}
                    for stat, stat_data in segment_stats.items():
                        stat_keys = stat_data.keys()
                        stat_columns = [f'{stat}_{col}' for col in stat_keys]
                        stat_values = stat_data.values()
                        _stat_dict = {k: v for k, v in zip(stat_columns, stat_values)}
                        segment_stats_dict.update(_stat_dict)
                    segment_dict.update(segment_stats_dict)
                    segment_dict['match_id'] = data_json['data']["attributes"]["id"]
                    player_round_damage_dict_list.append(segment_dict)
                
                elif segment['type'] == 'player-summary':
                    segment_dict = {}
                    attributes = segment['attributes']
                    segment_dict.update(attributes)
                    metadata = segment['metadata']
                    segment_dict.update(metadata)
                    segment_stats = segment['stats']
                    segment_stats_dict = {}
                    for stat, stat_data in segment_stats.items():
                        try:
                            stat_keys = stat_data.keys()
                            stat_columns = [f'{stat}_{col}' for col in stat_keys]
                            stat_values = stat_data.values()
                            _stat_dict = {k: v for k, v in zip(stat_columns, stat_values)}
                            segment_stats_dict.update(_stat_dict)
                        except AttributeError:
                            """Tratativa de excecao para o caso em que nao tivermos alguma coluna dentro dos status"""
                            pass
                    segment_dict.update(segment_stats_dict)
                    segment_dict['match_id'] = data_json['data']["attributes"]["id"]
                    player_summary_dict_list.append(segment_dict)

                elif segment['type'] == 'player-round-kills':
                    segment_dict = {}
                    attributes = segment['attributes']
                    segment_dict.update(attributes)
                    metadata = segment['metadata']
                    segment_metadata_dict = {}
                    for metadata_iter, metadata_data in metadata.items():
                        try:
                            metadata_keys = metadata_data.keys()
                            metadata_columns = [f'{metadata_iter}_{col}' for col in metadata_keys]
                            metadata_values = metadata_data.values()
                            _metadata_dict = {k: v for k, v in zip(metadata_columns, metadata_values)}
                            segment_metadata_dict.update(_metadata_dict)
                        except AttributeError:
                            """Tratativa de excecao para o caso em que nao tivermos alguma coluna dentro dos status"""
                            metadata_weaponImageUrl = metadata['weaponImageUrl']
                            metadata_weaponName = metadata['weaponName']
                            metadata_weaponCategory = metadata['weaponCategory']
                            metadata_gameTime = metadata['gameTime']
                            metadata_roundTime = metadata['roundTime']
                            pass
                        segment_stats_damage = segment['stats']['damage']
                        segment_metadata_dict.update(segment_stats_damage)
                    segment_dict.update(segment_metadata_dict)
                    segment_dict['match_id'] = data_json['data']["attributes"]["id"]
                    player_round_kills.append(segment_dict)

        return (metadata_dict_list, player_round_dict_list, 
                player_round_damage_dict_list, player_summary_dict_list, 
                player_round_kills)
                
import json

path_read = 'raw/trackergg/matches_report_details/'
path_write = 'cleaned/trackergg/matches_report_details/'

files = AwsS3.get_files_list(path_read)

data_json = json.loads(AwsS3.get_file(path_read, files[7].key))
data_json = json.dumps(data_json)

f = open("data_json.txt", "a")

f.write(data_json)
f.close()

In [5]:
df_details = pd.read_json('data_json.txt')
df_details

,data
attributes,{'id': '01d95487-8f75-47e0-8a39-6c01d9d8a5f0'}
expiryDate,2022-10-08T17:28:55.0478738+00:00
metadata,"{'modeKey': 'bomb', 'modeName': 'Competitive',..."
segments,"[{'type': 'team-summary', 'attributes': {'team..."


In [5]:
metadata_dict_list, player_round_dict_list, player_round_damage_dict_list, player_summary_dict_list, player_round_kills = DataCleaner.all_dfs()

In [5]:
df_match_metadata  = pd.DataFrame(metadata_dict_list)
df_player_round  = pd.DataFrame(player_round_dict_list)
df_player_round_damage  = pd.DataFrame(player_round_damage_dict_list)
df_player_summary  = pd.DataFrame(player_summary_dict_list)
df_player_round_kills  = pd.DataFrame(player_round_kills)



In [6]:
df_match_metadata.head(5)

,modeKey,modeName,modeImageUrl,modeMaxRounds,duration,dateStarted,rounds,isRanked,queueId,map,mapName,mapImageUrl,match_id
0,bomb,Competitive,https://trackercdn.com/cdn/tracker.gg/valorant...,25,2664018,2022-10-06T20:39:44.166+00:00,24,True,competitive,2c9d57ec-4431-9c5e-2939-8f9ef6dd5cba,Bind,https://trackercdn.com/cdn/tracker.gg/valorant...,b0406723-8338-48e7-83ca-164c0b74dd6f
1,bomb,Competitive,https://trackercdn.com/cdn/tracker.gg/valorant...,25,2026725,2022-10-04T21:38:03.822+00:00,19,True,competitive,fd267378-4d1d-484f-ff52-77821ed10dc2,Pearl,https://titles.trackercdn.com/valorant-api/map...,4bb1d39f-3808-4e4a-b655-136f0079b7d2
2,bomb,Competitive,https://trackercdn.com/cdn/tracker.gg/valorant...,25,1805823,2022-10-04T18:02:49.148+00:00,18,True,competitive,2bee0dc9-4ffe-519b-1cbd-7fbe763a6047,Haven,https://trackercdn.com/cdn/tracker.gg/valorant...,09a26011-4020-47cb-94dd-d9f1d1c35986
3,bomb,Competitive,https://trackercdn.com/cdn/tracker.gg/valorant...,25,2300029,2022-10-03T19:54:44.606+00:00,23,True,competitive,b529448b-4d60-346e-e89e-00a4c527a405,Fracture,https://titles.trackercdn.com/valorant-api/map...,5b93dbd4-266c-4426-82bc-c21ffcfdb65e
4,bomb,Competitive,https://trackercdn.com/cdn/tracker.gg/valorant...,25,2327494,2022-10-03T13:44:06.409+00:00,21,True,competitive,2c9d57ec-4431-9c5e-2939-8f9ef6dd5cba,Bind,https://trackercdn.com/cdn/tracker.gg/valorant...,0960e93b-9fbe-470f-88a2-3f64ffc074be
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,bomb,Competitive,https://trackercdn.com/cdn/tracker.gg/valorant...,25,1944986,2020-09-30T16:08:30.12+00:00,18,True,competitive,2c9d57ec-4431-9c5e-2939-8f9ef6dd5cba,Bind,https://trackercdn.com/cdn/tracker.gg/valorant...,83508218-74da-4213-83e2-589d2f1b9862
196,bomb,Competitive,https://trackercdn.com/cdn/tracker.gg/valorant...,25,1904097,2020-09-27T17:02:44.785+00:00,17,True,competitive,2c9d57ec-4431-9c5e-2939-8f9ef6dd5cba,Bind,https://trackercdn.com/cdn/tracker.gg/valorant...,e269dc82-3aa8-41a5-84af-d862e2eb94af
197,bomb,Competitive,https://trackercdn.com/cdn/tracker.gg/valorant...,25,2109072,2020-09-26T04:16:43.138+00:00,18,True,competitive,d960549e-485c-e861-8d71-aa9d1aed12a2,Split,https://trackercdn.com/cdn/tracker.gg/valorant...,3ea8b4ae-ae76-4a5b-9145-4d66ca48ba88
198,bomb,Competitive,https://trackercdn.com/cdn/tracker.gg/valorant...,25,2251814,2020-09-25T18:21:45.101+00:00,23,True,competitive,2bee0dc9-4ffe-519b-1cbd-7fbe763a6047,Haven,https://trackercdn.com/cdn/tracker.gg/valorant...,39be8b98-67c9-4169-92bb-abc5912f4a25


In [7]:
df_player_round

,round,platformSlug,platformUserIdentifier,teamId,teamSide,agentKey,platformInfo,hasWon,score_rank,score_percentile,...,spentCredits_rank,spentCredits_percentile,spentCredits_displayName,spentCredits_displayCategory,spentCredits_category,spentCredits_metadata,spentCredits_value,spentCredits_displayValue,spentCredits_displayType,match_id
0,1,riot,RayzenSama#6999,Blue,defender,320b2a48-4d9b-a075-30f1-1f93a9b638fa,"{'platformSlug': 'riot', 'platformUserId': Non...",False,None,None,...,None,None,Spent Credits,None,None,{},0,0,Number,b0406723-8338-48e7-83ca-164c0b74dd6f
1,1,riot,sou ruim mesmo#lol,Blue,defender,9f0d8ba9-4140-b941-57d3-a7ad57c6b417,"{'platformSlug': 'riot', 'platformUserId': Non...",False,None,None,...,None,None,Spent Credits,None,None,{},700,700,Number,b0406723-8338-48e7-83ca-164c0b74dd6f
2,1,riot,XXxbrunin777#8341,Red,attacker,22697a3d-45bf-8dd7-4fec-84a9e28c69d7,"{'platformSlug': 'riot', 'platformUserId': Non...",True,None,None,...,None,None,Spent Credits,None,None,{},750,750,Number,b0406723-8338-48e7-83ca-164c0b74dd6f
3,1,riot,Peido#3340,Blue,defender,f94c3b30-42be-e959-889c-5aa313dba261,"{'platformSlug': 'riot', 'platformUserId': Non...",False,None,None,...,None,None,Spent Credits,None,None,{},800,800,Number,b0406723-8338-48e7-83ca-164c0b74dd6f
4,1,riot,Kiunfps#5540,Red,attacker,f94c3b30-42be-e959-889c-5aa313dba261,"{'platformSlug': 'riot', 'platformUserId': Non...",True,None,None,...,None,None,Spent Credits,None,None,{},700,700,Number,b0406723-8338-48e7-83ca-164c0b74dd6f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40825,26,riot,iLt Deku#BR1,Red,defender,9f0d8ba9-4140-b941-57d3-a7ad57c6b417,"{'platformSlug': 'riot', 'platformUserId': Non...",True,None,None,...,None,None,Spent Credits,None,None,{},4600,"4,600",Number,ceb651b7-1155-4557-acd2-55d89a9bd1ca
40826,26,riot,rob12377#BR1,Blue,attacker,eb93336a-449b-9c1b-0a54-a891f7921d69,"{'platformSlug': 'riot', 'platformUserId': Non...",False,None,None,...,None,None,Spent Credits,None,None,{},4500,"4,500",Number,ceb651b7-1155-4557-acd2-55d89a9bd1ca
40827,26,riot,vamp peituda#6666,Red,defender,1e58de9c-4950-5125-93e9-a0aee9f98746,"{'platformSlug': 'riot', 'platformUserId': Non...",True,None,None,...,None,None,Spent Credits,None,None,{},4800,"4,800",Number,ceb651b7-1155-4557-acd2-55d89a9bd1ca
40828,26,riot,AfonsoTS#shlby,Red,defender,569fdd95-4d10-43ab-ca70-79becc718b46,"{'platformSlug': 'riot', 'platformUserId': Non...",True,None,None,...,None,None,Spent Credits,None,None,{},4400,"4,400",Number,ceb651b7-1155-4557-acd2-55d89a9bd1ca


In [8]:
df_player_round_damage

,round,platformSlug,platformUserIdentifier,opponentPlatformSlug,opponentPlatformUserIdentifier,damage_rank,damage_percentile,damage_displayName,damage_displayCategory,damage_category,...,headshots_rank,headshots_percentile,headshots_displayName,headshots_displayCategory,headshots_category,headshots_metadata,headshots_value,headshots_displayValue,headshots_displayType,match_id
0,1,riot,sou ruim mesmo#lol,riot,tks 痛風#6778,None,None,None,None,None,...,None,None,None,None,None,{},0,0,Number,b0406723-8338-48e7-83ca-164c0b74dd6f
1,1,riot,XXxbrunin777#8341,riot,sou ruim mesmo#lol,None,None,None,None,None,...,None,None,None,None,None,{},1,1,Number,b0406723-8338-48e7-83ca-164c0b74dd6f
2,1,riot,XXxbrunin777#8341,riot,Puruvudu#5914,None,None,None,None,None,...,None,None,None,None,None,{},1,1,Number,b0406723-8338-48e7-83ca-164c0b74dd6f
3,1,riot,XXxbrunin777#8341,riot,Peido#3340,None,None,None,None,None,...,None,None,None,None,None,{},1,1,Number,b0406723-8338-48e7-83ca-164c0b74dd6f
4,1,riot,XXxbrunin777#8341,riot,RayzenSama#6999,None,None,None,None,None,...,None,None,None,None,None,{},1,1,Number,b0406723-8338-48e7-83ca-164c0b74dd6f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52265,25,riot,iLt Deku#BR1,riot,Badona o trem#1234,None,None,None,None,None,...,None,None,None,None,None,{},1,1,Number,ceb651b7-1155-4557-acd2-55d89a9bd1ca
52266,25,riot,vamp peituda#6666,riot,ALIEN#PABLO,None,None,None,None,None,...,None,None,None,None,None,{},0,0,Number,ceb651b7-1155-4557-acd2-55d89a9bd1ca
52267,25,riot,AfonsoTS#shlby,riot,rob12377#BR1,None,None,None,None,None,...,None,None,None,None,None,{},0,0,Number,ceb651b7-1155-4557-acd2-55d89a9bd1ca
52268,25,riot,Badona o trem#1234,riot,The Big Best#0201,None,None,None,None,None,...,None,None,None,None,None,{},0,0,Number,ceb651b7-1155-4557-acd2-55d89a9bd1ca


In [9]:
df_player_summary

,platformSlug,platformUserIdentifier,partyId,teamId,agentKey,agentName,agentColor,agentImageUrl,agentPortraitUrl,countryCode,...,kast_rank,kast_percentile,kast_displayName,kast_displayCategory,kast_category,kast_metadata,kast_value,kast_displayValue,kast_displayType,match_id
0,riot,Peido#3340,cf87936b-3587-4d39-b94f-e33556e5da4e,Blue,f94c3b30-42be-e959-889c-5aa313dba261,Raze,#FF5589,https://titles.trackercdn.com/valorant-api/age...,https://trackercdn.com/cdn/tracker.gg/valorant...,None,...,None,None,None,None,None,{},79.166667,79.2%,NumberPercentage,b0406723-8338-48e7-83ca-164c0b74dd6f
1,riot,Puruvudu#5914,c2fefd14-3805-46ff-9bbf-7118090fceaa,Blue,bb2a4828-46eb-8cd1-e765-15848195d751,Neon,#2952A5,https://titles.trackercdn.com/valorant-api/age...,https://trackercdn.com/cdn/tracker.gg/valorant...,None,...,None,None,None,None,None,{},66.666667,66.7%,NumberPercentage,b0406723-8338-48e7-83ca-164c0b74dd6f
2,riot,tks 痛風#6778,f46360da-1baa-4bef-8671-d8558571d59c,Red,569fdd95-4d10-43ab-ca70-79becc718b46,Sage,#52FFD0,https://titles.trackercdn.com/valorant-api/age...,https://trackercdn.com/cdn/tracker.gg/valorant...,None,...,None,None,None,None,None,{},41.666667,41.7%,NumberPercentage,b0406723-8338-48e7-83ca-164c0b74dd6f
3,riot,Kiunfps#5540,f46360da-1baa-4bef-8671-d8558571d59c,Red,f94c3b30-42be-e959-889c-5aa313dba261,Raze,#FF5589,https://titles.trackercdn.com/valorant-api/age...,https://trackercdn.com/cdn/tracker.gg/valorant...,None,...,None,None,None,None,None,{},58.333333,58.3%,NumberPercentage,b0406723-8338-48e7-83ca-164c0b74dd6f
4,riot,iced coffee#1414,c4532f94-a009-494b-82e8-887a9672f6be,Red,a3bfb853-43b2-7238-a4f1-ad90e9e46bcc,Reyna,#f650d3,https://titles.trackercdn.com/valorant-api/age...,https://trackercdn.com/cdn/tracker.gg/valorant...,None,...,None,None,None,None,None,{},50.000000,50.0%,NumberPercentage,b0406723-8338-48e7-83ca-164c0b74dd6f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,riot,RayzenSama#6999,e264d74b-c1cd-48ee-b350-93f1d5ec03f4,Red,a3bfb853-43b2-7238-a4f1-ad90e9e46bcc,Reyna,#f650d3,https://titles.trackercdn.com/valorant-api/age...,https://trackercdn.com/cdn/tracker.gg/valorant...,None,...,None,None,None,None,None,{},69.230769,69.2%,NumberPercentage,ceb651b7-1155-4557-acd2-55d89a9bd1ca
1996,riot,rob12377#BR1,787cb956-8ea7-40cb-9d21-14e6cac24082,Blue,eb93336a-449b-9c1b-0a54-a891f7921d69,Phoenix,#FF6C40,https://titles.trackercdn.com/valorant-api/age...,https://trackercdn.com/cdn/tracker.gg/valorant...,None,...,None,None,None,None,None,{},57.692308,57.7%,NumberPercentage,ceb651b7-1155-4557-acd2-55d89a9bd1ca
1997,riot,isaveredice234#BR1,171737f2-97b9-4c74-b44a-bf99f12df9ee,Blue,a3bfb853-43b2-7238-a4f1-ad90e9e46bcc,Reyna,#f650d3,https://titles.trackercdn.com/valorant-api/age...,https://trackercdn.com/cdn/tracker.gg/valorant...,None,...,None,None,None,None,None,{},61.538462,61.5%,NumberPercentage,ceb651b7-1155-4557-acd2-55d89a9bd1ca
1998,riot,vamp peituda#6666,ec16eb85-9379-46d4-9fca-613a2b1c121f,Red,1e58de9c-4950-5125-93e9-a0aee9f98746,Killjoy,#FEC0CB,https://titles.trackercdn.com/valorant-api/age...,https://trackercdn.com/cdn/tracker.gg/valorant...,None,...,None,None,None,None,None,{},73.076923,73.1%,NumberPercentage,ceb651b7-1155-4557-acd2-55d89a9bd1ca


In [10]:
df_player_round_kills

,round,platformSlug,platformUserIdentifier,opponentPlatformSlug,opponentPlatformUserIdentifier,platformInfo_platformSlug,platformInfo_platformUserId,platformInfo_platformUserHandle,platformInfo_platformUserIdentifier,platformInfo_avatarUrl,...,opponentPlatformInfo_platformUserHandle,opponentPlatformInfo_platformUserIdentifier,opponentPlatformInfo_avatarUrl,opponentPlatformInfo_additionalParameters,opponentLocation_x,opponentLocation_y,finishingDamage_damageType,finishingDamage_damageItem,finishingDamage_isSecondaryFireMode,match_id
0,1,riot,XXxbrunin777#8341,riot,sou ruim mesmo#lol,riot,None,XXxbrunin777#8341,XXxbrunin777#8341,None,...,sou ruim mesmo#lol,sou ruim mesmo#lol,None,None,10963,2320,Weapon,856D9A7E-4B06-DC37-15DC-9D809C37CB90,True,b0406723-8338-48e7-83ca-164c0b74dd6f
1,1,riot,XXxbrunin777#8341,riot,Puruvudu#5914,riot,None,XXxbrunin777#8341,XXxbrunin777#8341,None,...,Puruvudu#5914,Puruvudu#5914,None,None,10240,641,Weapon,29A0CFAB-485B-F5D5-779A-B59F85E204A8,False,b0406723-8338-48e7-83ca-164c0b74dd6f
2,1,riot,XXxbrunin777#8341,riot,Peido#3340,riot,None,XXxbrunin777#8341,XXxbrunin777#8341,None,...,Peido#3340,Peido#3340,None,None,7526,1038,Weapon,1BAA85B4-4C70-1284-64BB-6481DFC3BB4E,False,b0406723-8338-48e7-83ca-164c0b74dd6f
3,1,riot,XXxbrunin777#8341,riot,RayzenSama#6999,riot,None,XXxbrunin777#8341,XXxbrunin777#8341,None,...,RayzenSama#6999,RayzenSama#6999,None,None,13560,-1296,Weapon,1BAA85B4-4C70-1284-64BB-6481DFC3BB4E,False,b0406723-8338-48e7-83ca-164c0b74dd6f
4,1,riot,XXxbrunin777#8341,riot,Bragets#BR1,riot,None,XXxbrunin777#8341,XXxbrunin777#8341,None,...,Bragets#BR1,Bragets#BR1,None,None,13163,-1633,Weapon,1BAA85B4-4C70-1284-64BB-6481DFC3BB4E,False,b0406723-8338-48e7-83ca-164c0b74dd6f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29317,26,riot,RayzenSama#6999,riot,CHINELO DE MÃE#HAHA,riot,None,RayzenSama#6999,RayzenSama#6999,None,...,CHINELO DE MÃE#HAHA,CHINELO DE MÃE#HAHA,None,None,4257,-4820,Weapon,29A0CFAB-485B-F5D5-779A-B59F85E204A8,False,ceb651b7-1155-4557-acd2-55d89a9bd1ca
29318,26,riot,CHINELO DE MÃE#HAHA,riot,vamp peituda#6666,riot,None,CHINELO DE MÃE#HAHA,CHINELO DE MÃE#HAHA,None,...,vamp peituda#6666,vamp peituda#6666,None,None,4735,-6943,Weapon,63E6C2B6-4A8E-869C-3D4C-E38355226584,True,ceb651b7-1155-4557-acd2-55d89a9bd1ca
29319,26,riot,iLt Deku#BR1,riot,ALIEN#PABLO,riot,None,iLt Deku#BR1,iLt Deku#BR1,None,...,ALIEN#PABLO,ALIEN#PABLO,None,None,4531,-5916,Weapon,9C82E19D-4575-0200-1A81-3EACF00CF872,True,ceb651b7-1155-4557-acd2-55d89a9bd1ca
29320,26,riot,vamp peituda#6666,riot,rob12377#BR1,riot,None,vamp peituda#6666,vamp peituda#6666,None,...,rob12377#BR1,rob12377#BR1,None,None,7511,-5834,Weapon,63E6C2B6-4A8E-869C-3D4C-E38355226584,True,ceb651b7-1155-4557-acd2-55d89a9bd1ca
